The following file tries to predict the w2v embeddings
from the EEG data.

First I'll try to predict using the raw data (without averaging).

In [1]:
import sklearn
import pandas as pd
import numpy as np
import pickle
import gensim
from numpy import savez_compressed
from numpy import load
import random

from sklearn.linear_model import Ridge
from sklearn.kernel_ridge import KernelRidge
# from sklearn.svm import LinearSVR
# from sklearn.svm import SVR
# from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA


In [2]:
readys_path = "Z:\\Jenn\\ml_df_readys.pkl"

# with open(pkl_path, 'rb') as f:
f = open(readys_path, 'rb')
readys_data = pickle.load(f)
f.close()


In [7]:
# sktime_path = "Z:\\Jenn\\ml_df_sktime.pkl"
#
# # with open(pkl_path, 'rb') as f:
# f1 = open(sktime_path, 'rb')
# sktime_data = pickle.load(f1)
# f1.close()

In [23]:
# No need to run this cell.
labels_mapping = {0:'baby', 1:'bear', 2:'bird', 3: 'bunny',
                  4:'cat', 5 : 'dog', 6: 'duck', 7: 'mom',
                  8: 'banana', 9: 'bottle', 10: 'cookie',
                  11: 'cracker', 12: 'cup', 13: 'juice',
                  14: 'milk', 15: 'spoon'}

In [27]:
# No need to run this cell.
# Extract the word labels first and store it locally.
def create_w2v_embeds():
    word_labels = readys_data['label'].values
    model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)
    # First obtain all the embeddings for the words in the labels mapping.
    w2v_label_embeds = {}
    pca = PCA(n_components=15)
    for key in labels_mapping:
        w2v_label_embeds[key] = pca.fit_transform([model[labels_mapping[key]]])
    all_embeds = []
    for label in word_labels:
        all_embeds.append(w2v_label_embeds[int(label)])
    savez_compressed('w2v_embeds/all_w2v_embeds_pcs.npz', all_embeds)

create_w2v_embeds()

ValueError: n_components=15 must be between 0 and min(n_samples, n_features)=1 with svd_solver='full'

In [17]:
eeg_features = readys_data.iloc[:,:18000].values
w2v_embeds_loaded = load('w2v_embeds/all_w2v_embeds.npz')
w2v_embeds = w2v_embeds_loaded['arr_0']

In [18]:
all_indices = [i for i in range(len(w2v_embeds))]
print(all_indices)
print(len(all_indices))
random.shuffle(all_indices)
print(all_indices)
eeg_features = [eeg_features[j] for j in all_indices]
eeg_features = np.array(eeg_features)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [19]:
# Split into training and testing data
X_train, X_test, y_train, y_test = train_test_split(eeg_features, w2v_embeds, test_size=0.1)
print(X_train.shape)
print(y_train.shape)
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)
# X_train = X_train.reshape(X_train.shape[0],-1)
# y_train = y_train.reshape(y_train.shape[0],300)
ridge = DecisionTreeRegressor()
ridge.fit(X_train, y_train)
preds = ridge.predict(X_test)

points = 0
total_points = 0
for i in range(preds.shape[0]-1):
    s_i = y_test[i]
    s_j = y_test[i+1]
    s_i_pred = preds[i]
    s_j_pred = preds[i+1]
    dsii = cosine_similarity([s_i], [s_i_pred])
    dsjj = cosine_similarity([s_j], [s_j_pred])
    dsij = cosine_similarity([s_i], [s_j_pred])
    dsji = cosine_similarity([s_j], [s_i_pred])
    if (dsii + dsjj) >= (dsij + dsji):
        points += 1
    total_points += 1
print( points / total_points)


(1553, 18000)
(1553, 300)
0.4883720930232558
